* Лекция 1. Введение  **<a href="https://www.youtube.com/watch?v=tIqndofykgc">YouTube</a>**  **<a href="https://vkvideo.ru/video-155161349_456239239?ref_domain=stepik.org">VK</a>**
* Лекция 2. Трюки: Deconvolution, Dilated Convolution  **<a href="https://www.youtube.com/watch?v=K73tZxH9nvE">YouTube</a>**  **<a href="https://vkvideo.ru/video-155161349_456239240?ref_domain=stepik.org">VK</a>**
* Лекция 3. Архитектура UNet  **<a href="https://www.youtube.com/watch?v=yEuIV5FsRMs">YouTube</a>**  **<a href="https://vkvideo.ru/video-155161349_456239241?ref_domain=stepik.org">VK</a>**
* Семинар 1. Семантическая сегментация **<a href="https://www.youtube.com/watch?v=QUYKIC7EmiM&ab_channel=DeepLearningSchool">YouTube</a>**  **<a href="https://vkvideo.ru/video-155161349_456239243?ref_domain=stepik.org">VK</a>**
* Семинар 2. Сегментация изображений с камеры беспилотника **<a href="https://www.youtube.com/watch?v=2zdbU7fUId8&ab_channel=%D0%90%D0%BA%D0%B0%D0%B4%D0%B5%D0%BC%D0%B8%D1%8F%D0%B8%D1%81%D0%BA%D1%83%D1%81%D1%81%D1%82%D0%B2%D0%B5%D0%BD%D0%BD%D0%BE%D0%B3%D0%BE%D0%B8%D0%BD%D1%82%D0%B5%D0%BB%D0%BB%D0%B5%D0%BA%D1%82%D0%B0">YouTube</a>**  **<a href="https://vkvideo.ru/video-155161349_456239242?ref_domain=stepik.org">VK</a>**

### 1. Введение

#### Постановка задачи
- Детекция - находим местоположение объекта на изображении
- Сегментация - понять к какому объекту принадлежит пиксель.
- Semantic - находим какие пиксели принадлежат конкретному классу объектов.
- Instance Segmentation - находим пиксели каждого объекта отдельно (вне зависимости от класса объекта)

<img src="./img/pws.png">

Мы должны заранее знать классы, на которые будем делить пиксели.
- Loss: Pixel-wise Softmax
- По сути сегментация это решение задачи классификации для каждого пикселя

#### Идеи решения: Sliding Windows

<img src="./img/sliding_window.png">

- Делим картинку на окна (совокупнуть пикселей) и прогоняем их через классификатор (например, ResNet)
- Минусы:
    - Нужно совершить много вызовов для классификации (вычисления)
    - Окна не шарят между собой о частях картинки (по маленькому окошку сложно будет определить, что это кусок коровы)

#### Идеи решения: Fully-conv network

<img src="./img/fully-conv.png" width=800>

Минусы:
- Большие конволюционные слои, очень много параметров. -> нужно сжимать информацию  о картинке перед построением карты сегментации

#### Идеи решения: CNN

<img src="./img/cnn.png" width=800>

- upsampling
- Минусы:
    - **Upsampling плохо восстанавливает информацию**
    - **Downsampling (макспулинг) и большой stride разрушают пространственную информацию (объект может быть в разных местах картинки)**
    - **Scale Variability (Вариативность масштаба) - подпроблема 2ого пункта. Объект может быть разных размеров - опять же сбивается местоположение**

### 2. Трюки: Deconvolution, Dilated Convolution
- **Решение проблемы Upsampling восстановления информации -> Deconvolution**
- Deconvolution делает feature map побольше.
- Как побороть проблема Upsampling -> Deconvolution (Unpooling)
- Transposed Convolution - говорить правильно, а не Deconvolution.

#### Transposed Convolution

<img src="./img/transposed_convolution.png" width=800>

- это upsampling + convolution

#### Виды upsampling

- Nearest Neighbours
<img src="./img/nearest.png">

- Bed of Nails

<img src="./img/bedon.png">

- Bilinear

<img src="./img/upsampling_bil.png">

- Max-Unpooling
    - нужно знать как мы делали пулинг в начале сети, чтобы сделать восстановление

<img src="./img/max_unpooling.png" width=750>

#### DeConvNet

<img src="./img/deconvnet.png" width=800>

- Сеть необязательно должна быть симметричной, однако в несимметричной сети применить Max-Unpooling нельзя.

#### Dilated Convolution
- Решение проблемы **Downsampling и большой stride разрушают пространственную
информацию** - Dilated Convolution
- Из фильтра 3х3 получили фильтр 5х5
- У Dilated convolution больший receptive field (тем самым захватываем больше пространства картинки)
- Заменяет stride

<img src="./img/dilated_conv.png" width=800>

- Заменяет пулинг?

<img src="./img/di_conv.png">

#### Multi-scale Context Aggregator

- Решение проблемы **Scale Variability**
- Применим к большому объекту Dilated Convolution с большим *R*, а к малому с меньшим *R*

<img src="./img/msca.png" width=600>

- Но мы не знаем какой R нужно применить -> поэтому делаем агрегатор делаем много параллельных Dilated Convolution с разными R.

<img src="./img/aggregator.png" width=800>

- Pyramid Pooling Network (распараллелим пулинги)

<img src="./img/ppn.png" width=800>


#### Вывод по трюкам

1. Deconvolution
2. Dilated Convolutions
3. Multi-scale Context Aggregator
4. Pyramid Pooling Network

### 3. Архитектура UNet

#### Segmentation Architectures
- SegNet (1)
- DeepLab v1 (2, 5)
- DeepLab v2 (2, 3, 5)
- DeepLab v3 (1, 2, 3, 5)
- DeepLab v3+ (1, 2, 3, 5)
- PSPNet (1, 2, 4)
- Mask R-CNN
- Это очень старое, лучше ПОЧЕКАТЬ НОВЫЕ.


#### UNet

<img src="./img/unet.png">

- Без Fully-connected слоев
- Стрелочки показывают куда течет информация
- Важно: есть Copy&concatenate (Skip connection)
- Симметричная
- Есть BottleNeck, похожа на бутылку
- Энкодер сжимает информацию, обработка
- Декодер восстанавливает пространственную информацию, а Skip-Connection помогает в этом.

#### Overlap-tile strategy

- Свертка убивает информацию о границе, поэтому сегментация на выходе может быть получена только для внутренней области изобржения -> В Unete решили это так, что сегментируется только то, что в красной рамки, а для того, что вне рамки - дополняем картинку отзеркаленным отображением (выделено коричневым). То есть зеркалим пограничные кусочки

<img src="./img/overlap.png" width=700>

#### UNet: Loss

- Есть тонкие линии на картинках (границы клеток, например). Проблема: как выделить границы, не относя к объектам?
- Для этого в UNet сделали веса. Даем большой loss(выделено красным). Пиксели между клетками - большой loss, большие веса.

<img src="./img/pwsoftmax.png" width=750>

---